In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models

from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
#os.system('python -m laserembeddings download-models')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=00f8c25806c6c1e0ff3bbc161c6fe93d9773d52610e643a1e1cdbccabc466a12
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
/kaggle/input/en-bn-sahih-bukhari-muslim/en_bn_translate

In [2]:
ls /opt/conda/lib/python3.7/site-packages/laserembeddings/data

In [3]:
import site
loc = site.getsitepackages() 
data_path = loc[0]+'/laserembeddings/data'
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/opt/conda/lib/python3.7/site-packages'

In [4]:

if not os.path.exists(data_path):
    os.makedirs('data')
import shutil
shutil.copy('../input/assets/93langs.fcodes', data_path)
shutil.copy('../input/assets/93langs.fvocab', data_path)
shutil.copy('../input/assets/bilstm.93langs.2018-12-26.pt', data_path)

!ls '/opt/conda/lib/python3.7/site-packages/laserembeddings/data'

93langs.fcodes	93langs.fvocab	bilstm.93langs.2018-12-26.pt


In [5]:
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/93langs.fcodes', './')
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/93langs.fvocab', './')
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/bilstm.93langs.2018-12-26.pt', './')


In [6]:
corpus_emb_quran = np.load('../input/assets/Holy_Quran_mlt_emb.npy')
corpus_emb_hadith = np.load('../input/assets/en_emb_bukhari_muslim.npy')

In [7]:
en_bn_quran_tafsir = pd.read_csv('../input/assets/en_bn_quran_tafsir.csv')

en_bn_bukhari_muslim = pd.read_csv('../input/assets/en_bn_bukhari_muslim.csv')

In [8]:
from numpy import dot
from numpy.linalg import norm
#https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)


def pairwise_dot(v1, v2):
    return (normalized(v1) * normalized(v2)).sum(axis=1)

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [9]:
en_bn_bukhari_muslim.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাত্তাবের বর্ণনায় উমর বিন আল-খাত্ত...


In [10]:
def MLT_Sahih_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot',query_embedding=None):

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = pairwise_dot(corpus_emb_hadith, query_embedding)

    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_hadith, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

    
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'source'] = str(en_bn_bukhari_muslim['source'][index])
        find.loc[i,'chapter_no'] = str(en_bn_bukhari_muslim['chapter_no'][index])
        find.loc[i,'hadith_no'] = str(en_bn_bukhari_muslim['hadith_no'][index])
        find.loc[i,'chapter'] = str(en_bn_bukhari_muslim['chapter'][index])
        find.loc[i,'text_ar'] = str(en_bn_bukhari_muslim['text_ar'][index])
        find.loc[i,'text_en'] = str(en_bn_bukhari_muslim['text_en'][index])
        find.loc[i,'text_bn'] = str(en_bn_bukhari_muslim['text_bn'][index])
        find.loc[i,'narrators'] = str(en_bn_bukhari_muslim['narrators'][index])
        

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore

    return find

In [11]:
en_bn_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...


In [12]:
def Multilingual_Quran_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    mlt_hadiths = MLT_Sahih_Hadith_Search_Engine(query,size=size,language = 'en',metric = metric,query_embedding=query_embedding)
    
    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = pairwise_dot(corpus_emb_quran, query_embedding)
   
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_quran, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

  
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'Name'] = str(en_bn_quran_tafsir['Name'][index])
        find.loc[i,'Surah'] = str(en_bn_quran_tafsir['Surah'][index])
        find.loc[i,'Ayat'] = str(en_bn_quran_tafsir['Ayat'][index])
        find.loc[i,'Verse'] = str(en_bn_quran_tafsir['Verse'][index]) 
        find.loc[i,'Tafseer'] = str(en_bn_quran_tafsir['Tafseer'][index]) 
        find.loc[i,'ar_text'] = str(en_bn_quran_tafsir['ar_text'][index])
        
        find.loc[i,'আল_বায়ান'] = str(en_bn_quran_tafsir['আল_বায়ান'][index])
        find.loc[i,'tafsir_bayan'] = str(en_bn_quran_tafsir['tafsir_bayan'][index])

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore
  
    return find,mlt_hadiths

In [13]:
%%time
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("fire",size=5,language = 'en',metric = 'dot')

CPU times: user 494 ms, sys: 75.8 ms, total: 570 ms
Wall time: 561 ms


In [14]:
find

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Event,56,94,And roasting at hell-fire.,and a roasting in Hell-fire.,وَّ تَصۡلِیَۃُ جَحِیۡمٍ ﴿۹۴﴾\nو تصلیۃ جحیم ﴿۹۴﴾,আর জ্বলন্ত আগুনে প্রজ্জ্বলনে।,(৯৪) এবং জাহান্নাম প্রবেশ।-তাফসীরে আহসানুল বায়ান,0.268898
1,Defrauding,83,9,A written record.,It is a sealed book.,کِتٰبٌ مَّرۡقُوۡمٌ ؕ﴿۹﴾\nکتب مرقوم ﴿۹﴾,লিখিত কিতাব।,৯। ওটা হচ্ছে লিপিবদ্ধ পুস্তক।-তাফসীরে আহসানুল ...,0.257667
2,The Event,56,43,"And shadow of black smoke,",and the shade of pitch-black smoke,وَّ ظِلٍّ مِّنۡ یَّحۡمُوۡمٍ ﴿ۙ۴۳﴾\nو ظل من یح...,"আর প্রচন্ড কালো ধোঁয়ার ছায়ায়,",(৪৩) কালোবর্ণ ধোঁয়ার ছায়ায়। [1][1] سَمُومٌ আগু...,0.244060
3,Solace,94,2,And eased thee of the burden,and relieve you of your burden,وَ وَضَعۡنَا عَنۡکَ وِزۡرَکَ ۙ﴿۲﴾\nو وضعنا عن...,"আর আমি নামিয়ে দিয়েছি তোমার থেকে তোমার বোঝা,",২। আমি তোমার উপর হতে অপসারণ করেছি তোমার সেই ভা...,0.237704
4,The Event,56,32,And fruit in plenty,and abundant fruit,وَّ فَاکِهَۃٍ کَثِیۡرَۃٍ ﴿ۙ۳۲﴾\nو فاکهۃ کثیرۃ...,"আর প্রচুর ফলমূলে,",(৩২) এবং প্রচুর ফলমূল;-তাফসীরে আহসানুল বায়ান,0.226537


In [15]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,77,6021,Dress,وكنت أغتسل أنا والنبي، صلى الله عليه وسلم من إ...,not found yet,এখনো পাওয়া যায়নি,Not Found,0.334566
1,Sahih Bukhari,77,5973,Dress,أو جابر بن عبد الله كان النبي صلى الله عليه وس...,not found yet,এখনো পাওয়া যায়নি,Not Found,0.334566
2,Sahih Bukhari,79,6359,Asking Permission,قالت ما كنت لأفشي على رسول الله صلى الله عليه ...,not found yet,এখনো পাওয়া যায়নি,Not Found,0.334415
3,Sahih Bukhari,68,5391,Divorce,قال حميد فقلت لزينب وما ترمي بالبعرة على رأس ا...,not found yet,এখনো পাওয়া যায়নি,Not Found,0.332493
4,Sahih Bukhari,68,5389,Divorce,قالت زينب فدخلت على زينب ابنة جحش حين توفي أخو...,not found yet,এখনো পাওয়া যায়নি,Not Found,0.332493


In [16]:
%%time
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("আমি কি পুরস্কার পেতে পারি?",size=3,language = 'en',metric = 'l2')
find

CPU times: user 407 ms, sys: 237 ms, total: 644 ms
Wall time: 199 ms


,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Pen,68,3,And lo! thine verily will be a reward unfailing.,And assuredly you will have an unfailing reward.,وَ اِنَّ لَکَ لَاَجۡرًا غَیۡرَ مَمۡنُوۡنٍ ۚ﴿۳...,আর নিশ্চয় তোমার জন্য রয়েছে অফুরন্ত পুরস্কার।,(৩) তোমার জন্য অবশ্যই রয়েছে নিরবচ্ছিন্ন পুরস্ক...,1.643042
1,The Star,53,59,"Marvel ye then at this statement,",Do you then marvel in denial at this discourse...,اَفَمِنۡ هٰذَا الۡحَدِیۡثِ تَعۡجَبُوۡنَ ﴿ۙ۵۹﴾...,তোমরা কি এ কথায় বিস্ময় বোধ করছ?,(৫৯) তোমরা কি এই কথায় বিস্ময়বোধ করছ? [1][1] এখ...,0.929772
2,The Calamity,101,10,"Ah, what will convey unto thee what she is! -",And what will show you what it is? that is to ...,وَ مَاۤ اَدۡرٰىکَ مَا هِیَهۡ ﴿ؕ۱۰﴾\nو ما ادرى...,আর তোমাকে কিসে জানাবে হাবিয়া কী?,"১০। কিসে তোমাকে জানাল, তা কি? [1][1] এখানে জাহ...",0.922465


In [17]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,66,5074,Virtues of the Qur'an,حدثنا محمد بن يوسف، حدثنا مالك بن مغول، حدثنا ...,Narrated Talha: I asked `...,তালেহা বর্ণিত: আমি `অবদুল্লাহ বিন আবী `আউফাকে ...,"Muhammad bin Yusuf al Baykandi ,Malik bin Mghw...",4.130790
1,Sahih Bukhari,39,2295,Kafalah,حدثنا أبو عاصم، عن يزيد بن أبي عبيد، عن سلمة ب...,Narrated Salama bin Al-Akwa`: ...,সালমা বিন আল-আকওয়া ` এর কাহিনীঃ একজন মৃত ব্যক...,"al Dahhak bin Mukhlad Abu 'Asim al Nabil ,Yazi...",4.097856
2,Sahih Muslim,13,2378,The Book of Zakat,وحدثنا عبد الله بن عبد الرحمن الدارمي، أخبرنا ...,This hadith has been narrated on the authorit...,এই হাদীসটি জায়েদ কর্তৃক একই কাতার সংরক্ষণকারী...,"'Abdullah bin 'Abdur Rahman al Daarimi ,Yahya ...",4.079525
